In [2]:
!pip install rarfile

In [6]:
import os
import rarfile
from google.colab import drive

# Mount Google Drive to access your files (only if your images are on Google Drive)
drive.mount('/content/drive')

# List of RAR file paths
archivos_rar = [
    '/content/drive/MyDrive/Proyecto/Indigenous.rar',
    '/content/drive/MyDrive/Proyecto/European.rar',
    '/content/drive/MyDrive/Proyecto/Afro-ecuadorians.rar',
    '/content/drive/MyDrive/Proyecto/Mestizos.rar',
]

# Base destination directory for extractions
directorio_destino_base = '/content/dataset_descomprimido'

# Process each RAR file in the list
for archivo_rar in archivos_rar:
    # Get the name of the file without extension
    nombre_sin_extension = os.path.splitext(os.path.basename(archivo_rar))[0]

    # Extract the RAR file to the destination directory
    with rarfile.RarFile(archivo_rar, 'r') as rar_ref:
        # Create the destination directory if it does not exist
        os.makedirs(os.path.join(directorio_destino_base, nombre_sin_extension), exist_ok=True)

        # Extract the contents of the RAR file to the destination directory
        for file_info in rar_ref.infolist():
            # Skip directories
            if file_info.is_dir():
                continue

            # Append only the base name of the extracted file to the destination path
            destino_archivo = os.path.join(directorio_destino_base, nombre_sin_extension, os.path.basename(file_info.filename))

            # Create the destination directory if it does not exist
            os.makedirs(os.path.dirname(destino_archivo), exist_ok=True)

            # Extract the file to the destination path
            with open(destino_archivo, 'wb') as f:
                f.write(rar_ref.read(file_info.filename))

        print(f"Extraction completed for: {nombre_sin_extension}")

print("Extraction of all RAR files completed.")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Extraction completed for: Indigenous
Extraction completed for: European
Extraction completed for: Afro-ecuadorians
Extraction completed for: Mestizos
Extraction of all RAR files completed.


In [7]:
import os
import cv2
import numpy as np
from google.colab import drive

# Montar Google Drive para acceder a tus archivos (solo si tus imágenes están en Google Drive)
drive.mount('/content/drive')

# Directorio base que contiene las carpetas descomprimidas
directorio_destino_base = '/content/dataset_descomprimido'

# Directorio base para guardar las imágenes umbralizadas
directorio_resultados_base = '/content/dataset_umbralizado'
os.makedirs(directorio_resultados_base, exist_ok=True)

def umbralizar(imagen):
    # Verificar si la imagen se leyó correctamente
    if imagen is None:
        print("No se pudo leer la imagen.")
        return None

    # Umbralizar y luego invertir los valores de los píxeles
    imagen_binarizada = np.where(imagen > 127, 255, 0).astype(np.uint8)
    return imagen_binarizada


# Procesar cada conjunto de imágenes descomprimidas
for carpeta in os.listdir(directorio_destino_base):
    # Reemplazar espacios con guiones bajos en el nombre de la carpeta
    carpeta_con_guiones = carpeta.replace(' ', '_')

    # Ruta completa de la carpeta actual
    directorio_actual = os.path.join(directorio_destino_base, carpeta)

    # Crear un directorio de resultados específico para la carpeta actual
    directorio_resultados_carpeta = os.path.join(directorio_resultados_base, carpeta_con_guiones)
    os.makedirs(directorio_resultados_carpeta, exist_ok=True)

    print(f"Procesando carpeta: {directorio_actual} -> {directorio_resultados_carpeta}")

    # Procesar cada imagen en la carpeta actual
    for archivo in os.listdir(directorio_actual):
        # Reemplazar espacios con guiones bajos en el nombre del archivo
        archivo_con_guiones = archivo.replace(' ', '_')

        # Leer la imagen en escala de grises
        ruta_imagen = os.path.join(directorio_actual, archivo)
        imagen = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)

        print(f"Leyendo imagen: {ruta_imagen}")

       # Aplicar umbralización invertida de Otsu solo si la imagen se leyó correctamente
        if imagen is not None:
            imagen_umbralizada = umbralizar(imagen)

            # Guardar la imagen umbralizada en el nuevo directorio
            if imagen_umbralizada is not None:
                ruta_resultado = os.path.join(directorio_resultados_carpeta, archivo_con_guiones)
                cv2.imwrite(ruta_resultado, imagen_umbralizada)
                print(f"Guardada imagen umbralizada invertida: {ruta_resultado}")
            else:
                print(f"No se pudo umbralizar la imagen: {ruta_imagen}")
        else:
            print(f"No se pudo leer la imagen: {ruta_imagen}")

print("Proceso de umbralización con Otsu completado para todos los conjuntos de imágenes.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Procesando carpeta: /content/dataset_descomprimido/European -> /content/dataset_umbralizado/European
Leyendo imagen: /content/dataset_descomprimido/European/002_7.JPG
Guardada imagen umbralizada invertida: /content/dataset_umbralizado/European/002_7.JPG
Leyendo imagen: /content/dataset_descomprimido/European/002_49.JPG
Guardada imagen umbralizada invertida: /content/dataset_umbralizado/European/002_49.JPG
Leyendo imagen: /content/dataset_descomprimido/European/002_11.JPG
Guardada imagen umbralizada invertida: /content/dataset_umbralizado/European/002_11.JPG
Leyendo imagen: /content/dataset_descomprimido/European/002_9.JPG
Guardada imagen umbralizada invertida: /content/dataset_umbralizado/European/002_9.JPG
Leyendo imagen: /content/dataset_descomprimido/European/002_1.JPG
Guardada imagen umbralizada invertida: /content/dataset_umbralizado/European/002_1.JPG
L

In [22]:
!pip install mahotas==1.4.12

import os
import cv2
import numpy as np
import mahotas
import pandas as pd
from google.colab import drive

# Montar Google Drive para acceder a tus archivos (solo si tus imágenes están en Google Drive)
drive.mount('/content/drive')

# Directorio base que contiene las carpetas descomprimidas
directorio_destino_base = '/content/dataset_descomprimido'

# Lista para almacenar los momentos de Zernike, HU y las etiquetas
momentos_zernike_total = []
momentos_hu_total = []
etiquetas_total = []

categorias = ['Afro-ecuadorians', 'European', 'Indigenous', 'Mestizos']

for categoria in categorias:
    # Procesar cada conjunto de imágenes descomprimidas para la categoría actual
    directorio_categoria = os.path.join(directorio_destino_base, categoria)
    for archivo in os.listdir(directorio_categoria):
        # Leer la imagen en escala de grises
        ruta_imagen = os.path.join(directorio_categoria, archivo)
        imagen = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)

        print(f"Leyendo imagen: {ruta_imagen}")

        # Calcular momentos de Zernike y HU solo si la imagen se leyó correctamente
        if imagen is not None:
            # Calcular los momentos de Zernike de la imagen
            momentos_z = mahotas.features.zernike_moments(imagen, 10, degree=6)
            momentos_zernike_total.append(momentos_z)

            # Calcular los momentos HU de la imagen
            moments_h = cv2.HuMoments(cv2.moments(imagen)).flatten()
            momentos_hu_total.append(moments_h)

            # Agregar etiqueta
            etiquetas_total.append(categoria)

# Convertir listas a arrays numpy
momentos_zernike_total = np.array(momentos_zernike_total)
momentos_hu_total = np.array(momentos_hu_total)
etiquetas_total = np.array(etiquetas_total)

# Crear DataFrame con los momentos de Zernike, HU y etiquetas
df = pd.DataFrame(momentos_zernike_total, columns=[f'Zernike_{i}' for i in range(momentos_zernike_total.shape[1])])
df_hu = pd.DataFrame(momentos_hu_total, columns=[f'HU_{i}' for i in range(momentos_hu_total.shape[1])])
df['Etiqueta'] = etiquetas_total

# Concatenar DataFrames de Zernike y HU
df_final = pd.concat([df, df_hu], axis=1)

# Guardar DataFrame en archivo CSV
archivo_csv = '/content/moments_total.csv'
df_final.to_csv(archivo_csv, index=False)

print(f"Todos los momentos calculados y guardados en CSV: {archivo_csv}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Leyendo imagen: /content/dataset_descomprimido/Afro-ecuadorians/003_48.JPG
Leyendo imagen: /content/dataset_descomprimido/Afro-ecuadorians/003_29.JPG
Leyendo imagen: /content/dataset_descomprimido/Afro-ecuadorians/003_17.JPG
Leyendo imagen: /content/dataset_descomprimido/Afro-ecuadorians/003_38.JPG
Leyendo imagen: /content/dataset_descomprimido/Afro-ecuadorians/003_44.JPG
Leyendo imagen: /content/dataset_descomprimido/Afro-ecuadorians/003_37.JPG
Leyendo imagen: /content/dataset_descomprimido/Afro-ecuadorians/003_1.JPG
Leyendo imagen: /content/dataset_descomprimido/Afro-ecuadorians/003_22.JPG
Leyendo imagen: /content/dataset_descomprimido/Afro-ecuadorians/003_2.JPG
Leyendo imagen: /content/dataset_descomprimido/Afro-ecuadorians/003_33.JPG
Leyendo imagen: /content/dataset_descomprimido/Afro-ecuadorians/003_4.JPG
Leyendo imagen: /content/dataset_descomprimido/Af

In [24]:
import os
import cv2
import numpy as np
import pandas as pd
from skimage.transform import resize
from skimage.feature import hog
from skimage import exposure
import matplotlib.pyplot as plt
from google.colab import drive

# Montar Google Drive para acceder a tus archivos (solo si tus imágenes están en Google Drive)
drive.mount('/content/drive')

# Directorio base que contiene las carpetas descomprimidas
directorio_destino_base = '/content/dataset_descomprimido'

# Lista de categorías
categorias = ['Afro-ecuadorians', 'European', 'Indigenous', 'Mestizos']

# Crear directorios para almacenar las características de HOG para cada categoría
directorio_hog_base = '/content/hog'
os.makedirs(directorio_hog_base, exist_ok=True)

# Lista para almacenar todas las características HOG y etiquetas
caracteristicas_hog_total = []
etiquetas_total = []

def calcular_caracteristicas_hog(imagen):
    # Redimensión de la imagen
    resized_img = resize(imagen, (128*4, 64*4))

    # Creación de características de HOG
    fd, hog_image = hog(resized_img, orientations=9, pixels_per_cell=(8, 8),
                        cells_per_block=(2, 2), visualize=True, multichannel=False)
    return fd

# Procesar cada conjunto de imágenes descomprimidas
for categoria in categorias:
    # Procesar cada conjunto de imágenes descomprimidas para la categoría actual
    directorio_categoria = os.path.join(directorio_destino_base, categoria)
    for archivo in os.listdir(directorio_categoria):
        # Leer la imagen en escala de grises
        ruta_imagen = os.path.join(directorio_categoria, archivo)
        imagen = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)

        print(f"Leyendo imagen: {ruta_imagen}")

        # Calcular características de HOG solo si la imagen se leyó correctamente
        if imagen is not None:
            # Calcular características de HOG
            caracteristicas = calcular_caracteristicas_hog(imagen)

            # Agregar características de HOG y etiqueta a las listas totales
            caracteristicas_hog_total.append(caracteristicas)
            etiquetas_total.append(categoria)

# Convertir listas a arrays numpy
caracteristicas_hog_total = np.array(caracteristicas_hog_total)
etiquetas_total = np.array(etiquetas_total)

# Crear DataFrame con características HOG y etiquetas
df = pd.DataFrame(caracteristicas_hog_total)
df['Etiqueta'] = etiquetas_total

# Guardar DataFrame en un solo archivo CSV
archivo_csv = os.path.join(directorio_hog_base, 'caracteristicas_hog_total.csv')
df.to_csv(archivo_csv, index=False)

print(f"Todas las características de HOG guardadas en el archivo CSV: {archivo_csv}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Leyendo imagen: /content/dataset_descomprimido/Afro-ecuadorians/003_48.JPG


<ipython-input-24-4b318c93c3ee>:33: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd, hog_image = hog(resized_img, orientations=9, pixels_per_cell=(8, 8),


Leyendo imagen: /content/dataset_descomprimido/Afro-ecuadorians/003_29.JPG
Leyendo imagen: /content/dataset_descomprimido/Afro-ecuadorians/003_17.JPG
Leyendo imagen: /content/dataset_descomprimido/Afro-ecuadorians/003_38.JPG
Leyendo imagen: /content/dataset_descomprimido/Afro-ecuadorians/003_44.JPG
Leyendo imagen: /content/dataset_descomprimido/Afro-ecuadorians/003_37.JPG
Leyendo imagen: /content/dataset_descomprimido/Afro-ecuadorians/003_1.JPG
Leyendo imagen: /content/dataset_descomprimido/Afro-ecuadorians/003_22.JPG
Leyendo imagen: /content/dataset_descomprimido/Afro-ecuadorians/003_2.JPG
Leyendo imagen: /content/dataset_descomprimido/Afro-ecuadorians/003_33.JPG
Leyendo imagen: /content/dataset_descomprimido/Afro-ecuadorians/003_4.JPG
Leyendo imagen: /content/dataset_descomprimido/Afro-ecuadorians/003_26.JPG
Leyendo imagen: /content/dataset_descomprimido/Afro-ecuadorians/003_7.JPG
Leyendo imagen: /content/dataset_descomprimido/Afro-ecuadorians/003_47.JPG
Leyendo imagen: /content/data

In [30]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Cargar el archivo CSV en un DataFrame de pandas
df = pd.read_csv('/content/hog/caracteristicas_hog_total.csv')

# Separar características (descriptores) y etiquetas
X = df.drop(columns=['Etiqueta'])  # Características
y = df['Etiqueta']  # Etiquetas

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar un modelo SVM con kernel RBF
svm_model = SVC(kernel='rbf')
svm_model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = svm_model.predict(X_test)

# Calcular la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


Accuracy: 0.5697674418604651
